# Testing EEG Model Classifications
Dataset https://www.kaggle.com/datasets/wajahat1064/emotion-recognition-using-eeg-and-computer-games/data

In [28]:
import pandas as pd
data = pd.read_csv("archive2/S01G1AllChannels.csv")
data.head(4)


,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Unnamed: 14
0,-33.0205,-15.1846,-42.1795,1.6872,42.1793,-1.68720,-5.5436,-3.6154,25.7899,-9.8819,5.5436,7.47180,11.8101,17.1128,NaN
1,-28.6291,-20.0583,-42.5410,-10.4653,35.3100,-15.68600,-19.3110,-2.4344,17.4933,3.2442,18.7081,5.09510,17.3683,3.0708,NaN
2,-21.8497,-10.9006,-32.0346,-2.3656,39.6993,-0.64483,-4.0523,-1.0830,26.8081,-3.4584,8.1861,8.40480,15.1209,9.3940,NaN
3,-25.1185,-10.9702,-32.7641,-3.4287,32.7378,4.69650,-8.6299,-1.7412,16.7637,-9.7586,1.1868,0.91086,4.3315,8.1073,NaN


# Clean data
Remove null values, etc.

# Visualize data
Tables for feature distribution, etc.


# Organize data
Create table by subject, game, and file path to EEG data

In [29]:
# 'camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G1AllRawChannels.csv'
data_list = []

for subject in range(28):
    subject = subject + 1
    if subject < 10:
        subject = f"0{subject}" 
    for game in range(4):
        arr = [subject, game + 1, f"camp_qmind/archive/data/data/GAMEEMO/(S{subject})/Raw EEG Data/.csv format/S{subject}G{game+1}AllRawChannels.csv"]
        data_list.append(arr)

data_list

[['01',
  1,
  'camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G1AllRawChannels.csv'],
 ['01',
  2,
  'camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G2AllRawChannels.csv'],
 ['01',
  3,
  'camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G3AllRawChannels.csv'],
 ['01',
  4,
  'camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G4AllRawChannels.csv'],
 ['02',
  1,
  'camp_qmind/archive/data/data/GAMEEMO/(S02)/Raw EEG Data/.csv format/S02G1AllRawChannels.csv'],
 ['02',
  2,
  'camp_qmind/archive/data/data/GAMEEMO/(S02)/Raw EEG Data/.csv format/S02G2AllRawChannels.csv'],
 ['02',
  3,
  'camp_qmind/archive/data/data/GAMEEMO/(S02)/Raw EEG Data/.csv format/S02G3AllRawChannels.csv'],
 ['02',
  4,
  'camp_qmind/archive/data/data/GAMEEMO/(S02)/Raw EEG Data/.csv format/S02G4AllRawChannels.csv'],
 ['03',
  1,
  'camp_qmind/archive/data/data/GAMEEMO/(S03)/Raw EEG Data/.csv format/S03G1AllRawChannels.csv'],
 

In [30]:

df = pd.DataFrame(data_list, columns = ['Subject', 'Game', 'Path']) 
df

,Subject,Game,Path
0,01,1,camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw...
1,01,2,camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw...
2,01,3,camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw...
3,01,4,camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw...
4,02,1,camp_qmind/archive/data/data/GAMEEMO/(S02)/Raw...
...,...,...,...
107,27,4,camp_qmind/archive/data/data/GAMEEMO/(S27)/Raw...
108,28,1,camp_qmind/archive/data/data/GAMEEMO/(S28)/Raw...
109,28,2,camp_qmind/archive/data/data/GAMEEMO/(S28)/Raw...
110,28,3,camp_qmind/archive/data/data/GAMEEMO/(S28)/Raw...


In [36]:
# /Users/mercy/Downloads/eeg/camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G1AllRawChannels.csv
# /Users/mercy/Downloads/eeg/camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G0AllRawChannels.csv

path = df.iloc[0]['Path']
print(path)

data = pd.read_csv('../' + path)
data.head(4)

camp_qmind/archive/data/data/GAMEEMO/(S01)/Raw EEG Data/.csv format/S01G1AllRawChannels.csv


,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Unnamed: 14
0,-35.1282,-16.1538,-44.8718,1.7949,44.8716,-1.7949,-5.8975,-3.8462,27.4360,-10.5127,5.8975,7.9487,12.5640,18.2051,NaN
1,-32.5642,-22.3079,-47.9487,-11.0256,40.2561,-16.7949,-20.8975,-2.8206,20.2561,2.8206,20.2561,5.8972,19.2307,4.3591,NaN
2,-27.1794,-13.8459,-39.4871,-3.0769,47.1794,-1.7949,-5.8975,-1.5383,31.2820,-4.1028,10.2566,9.7434,17.9485,11.2820,NaN
3,-32.0515,-14.6157,-42.3079,-4.3589,42.3076,3.8462,-11.0259,-2.3076,22.3076,-11.0259,3.3330,2.3076,7.4355,10.5127,NaN
